In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

from helper_functions import unzip_data, plot_loss_curves, compare_historys

!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

unzip_data("nlp_getting_started.zip")

--2024-08-28 04:17:52--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-08-28 04:17:53 (50.2 MB/s) - ‘helper_functions.py’ saved [10246/10246]

--2024-08-28 04:17:55--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.0.91, 172.217.7.59, 172.217.12.27, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.0.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application

In [ ]:
import pandas as pd

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_df_shuffled = train_df.sample(frac =1 , random_state= 1234) #隨機打亂train_df的row順序

from sklearn.model_selection import  train_test_split

#資料已被分為訓練集和測試集，因此這裡是從訓練集中，切驗證集出來
train_sentences , val_sentences , train_labels , val_labels = train_test_split(train_df_shuffled['text'].to_numpy() ,
                                            train_df_shuffled['target'].to_numpy(),
                                            test_size = 0.3 ,
                                            random_state = 1234)
#訓練集自變項、驗證集自變項、訓練集依變項、驗證集依變項
train_sentences.shape , val_sentences.shape , train_labels.shape , val_labels.shape

((5329,), (2284,), (5329,), (2284,))

In [ ]:
from tensorflow.keras.layers import TextVectorization
max_vocab_length = 10000 #詞彙的最大數
max_length = 15  #15個單字的長度

text_vectorizer = TextVectorization(max_tokens = max_vocab_length ,
                   output_mode = 'int',
                   output_sequence_length = max_length)

text_vectorizer.adapt(train_sentences) #建立訓練集

from tensorflow.keras import layers

embedding = layers.Embedding(input_dim= max_vocab_length ,
                output_dim = 128 ,
                embeddings_initializer = 'uniform') #均勻分布之初始化方法

In [ ]:
#model_1
model_1 = tf.keras.Sequential([
    layers.Input(shape = (1, ) , dtype = 'string'), #shape = (1,)表示資料的型態為一維，且每次輸入一個文本
    text_vectorizer ,
    embedding ,
    layers.GlobalAveragePooling1D() , #有助於減少維度並將資訊匯總為一個單一的特徵向量
    layers.Dense(1, activation = 'sigmoid')
    ])

model_1.compile(loss = tf.keras.losses.BinaryCrossentropy() ,
                optimizer = tf.keras.optimizers.Adam() ,
                metrics = ['accuracy'])

model_1_history =  model_1.fit(train_sentences,
                 train_labels,
                 epochs=5,
                 validation_data=(val_sentences, val_labels))

Epoch 1/5
167/167 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - accuracy: 0.6308 - loss: 0.6592 - val_accuracy: 0.7592 - val_loss: 0.5578
Epoch 2/5
167/167 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.7986 - loss: 0.5027 - val_accuracy: 0.7995 - val_loss: 0.4668
Epoch 3/5
167/167 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.8468 - loss: 0.3909 - val_accuracy: 0.8012 - val_loss: 0.4446
Epoch 4/5
167/167 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.8829 - loss: 0.3105 - val_accuracy: 0.7990 - val_loss: 0.4384
Epoch 5/5
167/167 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9184 - loss: 0.2501 - val_accuracy: 0.7986 - val_loss: 0.4445


In [ ]:
#model_2-LSTM
model_2 = tf.keras.Sequential([
    layers.Input(shape = (1, ) , dtype = 'string'),
    text_vectorizer ,
    embedding ,
    layers.LSTM(64), #處理序列資料，能找出文本中的依賴關係
    #layers.GlobalAveragePooling1D() ,
    layers.Dense(1, activation = 'sigmoid')
    ])

model_2.compile(loss = tf.keras.losses.BinaryCrossentropy() ,
                optimizer = tf.keras.optimizers.Adam() ,
                metrics = ['accuracy'])

model_2_history =  model_1.fit(train_sentences,
                 train_labels,
                 epochs=5,
                 validation_data=(val_sentences, val_labels))

Epoch 1/5
167/167 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9297 - loss: 0.2163 - val_accuracy: 0.8004 - val_loss: 0.4626
Epoch 2/5
167/167 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9410 - loss: 0.1746 - val_accuracy: 0.7925 - val_loss: 0.4802
Epoch 3/5
167/167 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.9555 - loss: 0.1453 - val_accuracy: 0.7964 - val_loss: 0.5004
Epoch 4/5
167/167 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9618 - loss: 0.1270 - val_accuracy: 0.7872 - val_loss: 0.5231
Epoch 5/5
167/167 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9647 - loss: 0.1117 - val_accuracy: 0.7881 - val_loss: 0.5519


In [ ]:
model_5 = tf.keras.Sequential([
    layers.Input(shape = (1, ) , dtype = 'string'),
    text_vectorizer ,
    embedding ,
    layers.Conv1D(32 , 5 , activation= 'relu'), #藉由32個大小為5的卷積核進行特徵選取，找出文本中的關係
    layers.GlobalMaxPool1D() , #從每個特徵圖中選擇最大值
    layers.Dense(1, activation = 'sigmoid')
    ])

model_5.compile(loss = tf.keras.losses.BinaryCrossentropy() ,
                optimizer = tf.keras.optimizers.Adam() ,
                metrics = ['accuracy'])

model_5_history =  model_1.fit(train_sentences,
                 train_labels,
                 epochs=5,
                 validation_data=(val_sentences, val_labels))

Epoch 1/5
167/167 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.9669 - loss: 0.1019 - val_accuracy: 0.7767 - val_loss: 0.5746
Epoch 2/5
167/167 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.9764 - loss: 0.0856 - val_accuracy: 0.7824 - val_loss: 0.5980
Epoch 3/5
167/167 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9746 - loss: 0.0785 - val_accuracy: 0.7811 - val_loss: 0.6178
Epoch 4/5
167/167 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9788 - loss: 0.0675 - val_accuracy: 0.7771 - val_loss: 0.6415
Epoch 5/5
167/167 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.9798 - loss: 0.0626 - val_accuracy: 0.7715 - val_loss: 0.6705


In [ ]:
train_10_percent_split = int(0.1 * len(train_sentences)) #從訓練樣本中提取10%的樣本數量，將結果轉換為整數，以確保分割點是整數值
print(train_10_percent_split)

train_sentences_10_percent = train_sentences[:train_10_percent_split] #從train_sentences中提取前10%的樣本

train_labels_10_percent = train_labels[:train_10_percent_split]

len(train_labels_10_percent) , train_labels_10_percent[:5] #len表示，計算train_labels_10_percent的長度，即標籤數量；取train_labels_10_percent中的前5個標籤

532


(532, array([0, 1, 1, 1, 0]))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

baseline_model = Pipeline([('tfidf' , TfidfVectorizer()), #Tf表示詞語在文本中出現的頻率；詞語在文本中的稀有程度，越稀有idf值越高
               ('clf' , MultinomialNB()),]) #使用朴素貝葉斯分類器

baseline_model.fit(train_sentences_10_percent , train_labels_10_percent)

#評估baseline_model
baseline_score = baseline_model.score(val_sentences , val_labels)
baseline_score #正確率

0.7373029772329247

In [ ]:
pip install tf_keras

In [ ]:
import tf_keras

In [ ]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [ ]:
import tensorflow_hub as hub
use_model = tf_keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", #USE是已經訓練好的模型，於此是進行選取文本的特徵
                   input_shape=[],
                   dtype=tf.string,
                   trainable=True),
    tf_keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
use_model.compile(loss="binary_crossentropy",
                optimizer=tf_keras.optimizers.Adam(),
                metrics=["accuracy"])

use_model_history = use_model.fit(train_sentences_10_percent,
                  train_labels_10_percent,
                  epochs=5,
                  validation_data=(val_sentences, val_labels))

Epoch 1/5
17/17 [==============================] - 122s 6s/step - loss: 0.6449 - accuracy: 0.6504 - val_loss: 0.5922 - val_accuracy: 0.7627
Epoch 2/5
17/17 [==============================] - 108s 6s/step - loss: 0.5045 - accuracy: 0.8252 - val_loss: 0.5267 - val_accuracy: 0.7574
Epoch 3/5
 9/17 [==============>...............] - ETA: 46s - loss: 0.3701 - accuracy: 0.9132

KeyboardInterrupt: 

In [ ]:
whole_train_sentences = train_df_shuffled['text'].to_numpy()
whole_train_labels =  train_df_shuffled['target'].to_numpy()

len(whole_train_sentences) , len(whole_train_labels)

(7613, 7613)

In [ ]:
model = tf_keras.Sequential([
  hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                   input_shape=[],
                   dtype=tf.string,
                   trainable=True),
  tf_keras.layers.Dense(64 , activation ='relu'),
  tf_keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.compile(loss="binary_crossentropy",
                optimizer=tf_keras.optimizers.Adam(),
                metrics=["accuracy"])

model_history =model.fit(whole_train_sentences,
              whole_train_labels,
              epochs=5,
              validation_data=(val_sentences, val_labels))

Epoch 1/5
238/238 [==============================] - 336s 1s/step - loss: 0.4467 - accuracy: 0.8059 - val_loss: 0.2618 - val_accuracy: 0.9094
Epoch 2/5
238/238 [==============================] - 320s 1s/step - loss: 0.2328 - accuracy: 0.9086 - val_loss: 0.0834 - val_accuracy: 0.9742
Epoch 3/5
238/238 [==============================] - 320s 1s/step - loss: 0.0766 - accuracy: 0.9744 - val_loss: 0.0312 - val_accuracy: 0.9908
Epoch 4/5
238/238 [==============================] - 320s 1s/step - loss: 0.0409 - accuracy: 0.9863 - val_loss: 0.0202 - val_accuracy: 0.9943
Epoch 5/5
238/238 [==============================] - 320s 1s/step - loss: 0.0270 - accuracy: 0.9905 - val_loss: 0.0207 - val_accuracy: 0.9921


In [ ]:
pred_probs = model.predict(test_df['text'].to_numpy())

preds = tf.squeeze(tf.round(pred_probs)) #將預測機率轉換為整數類別（０或１），並去不必要的維度
preds = tf.cast(preds , dtype = tf.int32) #確定預測資料型態為32位元

NameError: name 'model' is not defined

In [ ]:
test_df

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [ ]:
submission = pd.DataFrame({'id': test_df['id'].values ,
               'target': (preds.numpy())})
submission

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1


In [ ]:
preds[:20]

<tf.Tensor: shape=(20,), dtype=int32, numpy=
array([1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
      dtype=int32)>

In [ ]:
submission.to_csv('submission.csv' , index = False) #表示不儲存資料中的索引欄

In [ ]:
test_sentences = test_df['text'].to_numpy()

test_sentences.shape

(3263,)

In [ ]:
def give_preds(model, test_sentences):

  pred_probs = model.predict(test_sentences)
  preds = tf.squeeze(tf.round(pred_probs))
  preds = tf.cast(preds , dtype = tf.int32)

  return preds

In [ ]:
model_preds = give_preds(model , test_sentences)
model_2_preds = give_preds(model_2 ,test_sentences)
model_5_preds = give_preds(model_5 , test_sentences)

model_preds.shape , model_2_preds.shape , model_5_preds.shape

102/102 [==============================] - 0s 2ms/step


(TensorShape([3263]), TensorShape([3263]), TensorShape([3263]))

In [ ]:
model_preds[:5] , model_2_preds[:5] , model_5_preds[:5]

(<tf.Tensor: shape=(5,), dtype=int32, numpy=array([1, 1, 1, 1, 1], dtype=int32)>,
 <tf.Tensor: shape=(5,), dtype=int32, numpy=array([1, 1, 0, 1, 1], dtype=int32)>,
 <tf.Tensor: shape=(5,), dtype=int32, numpy=array([0, 0, 0, 0, 0], dtype=int32)>)

In [ ]:
preds_df = pd.DataFrame({'USE_model': model_preds ,
              'LSTM_model': model_2_preds ,
              'Conv1D_model': model_5_preds})
preds_df.head()

,USE_model,LSTM_model,Conv1D_model
0,1,1,0
1,1,1,0
2,1,0,0
3,1,1,0
4,1,1,0


In [ ]:
# Majority voted predictions of our model
preds_df['majority_vote_preds'] = preds_df.mode(axis = 1) #從每一row中，選出最頻率最高的值，當最終值，取名為majority_vote_preds
preds_df.head()

,USE_model,LSTM_model,Conv1D_model,majority_vote_preds
0,1,1,0,1
1,1,1,0,1
2,1,0,0,0
3,1,1,0,1
4,1,1,0,1


In [ ]:
mode_preds_array = preds_df['majority_vote_preds'].to_numpy()
mode_preds_array

array([1, 1, 0, ..., 1, 1, 1], dtype=int32)

In [ ]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15, norm=False, savefig=False):

  cm = confusion_matrix(y_true, y_pred) #row表示真實的類別、column表示預測的類別
  cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] #np.newaxis表示增加一個維度
  n_classes = cm.shape[0] #取得混淆矩陣的row數，也就是分類問題中的類別數

  fig, ax = plt.subplots(figsize=figsize)
  cax = ax.matshow(cm, cmap=plt.cm.Blues) #繪製混淆矩陣
  fig.colorbar(cax) #顯示顏色與數值的對應關係

  if classes:
    labels = classes
  else:
    labels = np.arange(cm.shape[0]) #顯示混淆矩陣的標籤

  ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes),
         yticks=np.arange(n_classes),
         xticklabels=labels,
         yticklabels=labels)

  ax.xaxis.set_label_position("bottom") #設置x軸的標籤在底部
  ax.xaxis.tick_bottom() #設置x軸的刻度在底部

  plt.xticks(rotation=70, fontsize=text_size) #避免x軸標籤彼此重疊
  plt.yticks(fontsize=text_size)

  threshold = (cm.max() + cm.min()) / 2. #確定閾值進行資料中顏色的範圍

  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):#分別表示混淆矩陣的row、column
    if norm: #norm表示是否使用標準化的混淆矩陣；為真
      plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)
    else: #為否
      plt.text(j, i, f"{cm[i, j]}",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)

  if savefig:
    fig.savefig("confusion_matrix.png")

In [ ]:
val_preds = give_preds(model , test_sentences)

class_names = ['not_a_disaster' , 'disaster']

val_preds.shape , test_labels.shape

102/102 [==============================] - 1s 6ms/step


(TensorShape([3263]), (2284,))

In [ ]:
make_confusion_matrix(y_true= test_labels ,
           y_pred = val_preds ,
           classes = class_names ,
           figsize= (15 , 7),
           norm = False ,
           savefig = True ,
           text_size = 15)

ValueError: Found input variables with inconsistent numbers of samples: [2284, 3263]